## Directory parameters

In [24]:
# Update working directory

%cd /home/hugo/Bureau/PoincareMSA/

/home/hugo/Bureau/PoincareMSA


In [25]:
import sys

# Add the project root to Python path
project_root = "/home/hugo/Bureau/PoincareMSA"
if project_root not in sys.path:
    sys.path.append(project_root)

## Librairies

In [26]:
import os
import itertools
import pandas as pd
import torch   # ⬅️ nécessaire pour charger les embeddings AAE (.pt)

# Fonctions internes déjà existantes
from scripts.visualize_projection.pplots_new import (
    read_embeddings,
    plot_embedding_interactive
)

from scripts.visualize_projection.pplots_new import read_embeddings, plot_embedding_interactive
from scripts.build_poincare_map.poincare_maps import PoincareMaps
from scripts.build_poincare_map.embedding_quality_score import get_quality_metrics
from sklearn.cluster import AgglomerativeClustering, SpectralClustering
from sklearn.metrics import adjusted_rand_score, fowlkes_mallows_score
from scipy import stats
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


## Parameters

In [27]:
data_type = "plm_aae"       # "plm", "plm_aae", "pssm", "RFA_matrix"

# Annotation CSV
path_annotation_csv = "examples/kinases/kinase_group_new.csv"

# Embeddings or fasta folder depending on type
embedding_paths = {
    "plm":       "embeddings/ankh_base_kinases/",
    "plm_aae":   "embeddings/aae_embeddings/ankh_base_kinases/",
    "pssm":      "examples/kinases/glob.mfasta",
    "RFA_matrix": None
}

# Output bases
output_base = "grid_search_results/" + data_type
os.makedirs(output_base, exist_ok=True)

In [28]:
grid = {
    "knn":       [5],
    "gamma":     [0.5, 1],
    "sigma":     [1],
    "cospca":    [0],
    "epochs":    [10],
    "seed":      [0,1,2],
    "distance":  ["cosine"],
}

## Usefull functions

In [29]:
def get_scores(true_labels, pred_labels):
    ARI = adjusted_rand_score(true_labels, pred_labels)
    FMS = fowlkes_mallows_score(true_labels, pred_labels)
    return ARI, FMS

def detect_cluster(distances, n_clusters=2, clustering_name='agglomerative'):
    if clustering_name == 'agglomerative':
        clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='average').fit(distances)
        labels = clustering.labels_
    return labels


In [30]:
def build_command(params, folder_output):
    project_root = "/home/hugo/Bureau/PoincareMSA"

    if data_type in ["plm", "plm_aae"]:
        input_path = embedding_paths[data_type]
        plm_flag = "True"
        mid_output = folder_output + "/matrices/"
        os.makedirs(mid_output, exist_ok=True)
    elif data_type == "pssm":
        # Préparer les données PSSM si nécessaire
        pssm_prep_path = "kinases_data/fasta0.9"
        if not os.path.exists(pssm_prep_path):
            os.system(f"bash scripts/prepare_data/create_projection.sh scripts/prepare_data examples/kinases/glob.mfasta kinases_data 0.9")

        input_path = pssm_prep_path
        plm_flag = "False"
        mid_output = folder_output + "/matrices/"
        os.makedirs(mid_output, exist_ok=True)
    elif data_type == "RFA_matrix":
        input_path = "None"
        plm_flag = "True"
        mid_output = "RFA_matrix/kinases/with_plm_embeddings/"

    cmd = (
        f"PYTHONPATH={project_root}:$PYTHONPATH python scripts/build_poincare_map/main.py "
        f"--input_path {input_path} "
        f"--output_path {folder_output} "
        f"--plm_embedding {plm_flag} "
        f"--matrices_output_path {mid_output} "
        f"--distlocal {params['distance']} "
        f"--gamma {params['gamma']} "
        f"--pca {params['cospca']} "
        f"--epochs {params['epochs']} "
        f"--seed {params['seed']} "
        f"--knn {params['knn']} "
    )
    return cmd

In [31]:
def load_projection(folder_output, params):
    # Essayer plusieurs emplacements possibles
    possible_files = [
        f"{folder_output}/PM{params['knn']}sigma={params['sigma']:.2f}gamma={params['gamma']:.2f}{params['distance']}pca={params['cospca']}_seed{params['seed']}.csv",
        f"{folder_output}/matrices/PM{params['knn']}sigma={params['sigma']:.2f}gamma={params['gamma']:.2f}{params['distance']}pca={params['cospca']}_seed{params['seed']}.csv"
    ]

    for file in possible_files:
        if os.path.exists(file):
            return file

    # Si aucun fichier n'est trouvé, retourner le chemin attendu
    return possible_files[0]


In [32]:
def compare_projections(results):
    # Convert results to DataFrame
    df_results = pd.DataFrame(results)

    # Create a new column that excludes the seed from the parameter string
    df_results["params_base"] = df_results["params"].apply(
        lambda x: "_".join([f"{k}{v}" for k, v in x.items() if k != "seed"])
    )

    # Add seed as a separate column for visualization
    df_results["seed"] = df_results["params"].apply(lambda x: x["seed"])

    # Print available metrics
    print("\n=== Comparison of Projections ===")
    available_cols = ["params_base", "seed", "ARI", "FMS"]
    if "Qlocal" in df_results.columns and df_results["Qlocal"].notna().any():
        available_cols.append("Qlocal")
    if "Qglobal" in df_results.columns and df_results["Qglobal"].notna().any():
        available_cols.append("Qglobal")
    print(df_results[available_cols])

    # Plot ARI if available
    if "ARI" in df_results.columns and df_results["ARI"].notna().any():
        fig_ari = px.strip(
            df_results,
            x="params_base",
            y="ARI",
            color="seed",  # Color by seed to distinguish different runs
            title="ARI by parameter combination (colored by seed)"
        )
        fig_ari.update_layout(
            xaxis_title="Parameter combination (excluding seed)",
            yaxis_title="ARI",
            legend_title="Seed"
        )
        fig_ari.show()

    # Plot FMS if available
    if "FMS" in df_results.columns and df_results["FMS"].notna().any():
        fig_fms = px.strip(
            df_results,
            x="params_base",
            y="FMS",
            color="seed",  # Color by seed to distinguish different runs
            title="FMS by parameter combination (colored by seed)"
        )
        fig_fms.update_layout(
            xaxis_title="Parameter combination (excluding seed)",
            yaxis_title="FMS",
            legend_title="Seed"
        )
        fig_fms.show()

    # Plot Qnx if available
    if "Qnx" in df_results.columns and any(len(qnx) > 0 for qnx in df_results["Qnx"]):
        # Create a palette for parameter combinations
        param_palette = dict(zip(
            df_results["params_base"].unique(),
            sns.color_palette("husl", n_colors=len(df_results["params_base"].unique())).as_hex()
        ))

        # Create a palette for seeds
        seed_palette = sns.color_palette("pastel", n_colors=len(df_results["seed"].unique()))

        plt.figure(figsize=(14, 7))

        # Group by parameter combination
        for param_base, param_group in df_results.groupby("params_base"):
            # For each seed in this parameter group
            for _, row in param_group.iterrows():
                if isinstance(row["Qnx"], (list, np.ndarray)) and len(row["Qnx"]) > 0:
                    seed = row["seed"]
                    # Use a different line style for each seed
                    linestyle = '-' if seed == param_group["seed"].min() else '--'
                    sns.lineplot(
                        x=range(1, len(row["Qnx"]) + 1),
                        y=row["Qnx"],
                        color=param_palette[param_base],
                        linestyle=linestyle,
                        label=f"{param_base} (seed={seed})",
                        linewidth=2 if seed == param_group["seed"].min() else 1
                    )

        plt.xlabel("K")
        plt.ylabel("Qnx")
        plt.ylim([0, 1.1])
        plt.title("Qnx by parameter combination (different seeds)")
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()


## Grid search & plot

In [33]:
# %%
print("=== Starting grid search ===")
df_annotation = pd.read_csv(path_annotation_csv)

# List to store the results of projections
results = []

# Define a custom color palette for visualization
kinase_palette = {-1 : "#c7c7c7", "OTHER": "#c7c7c7", "None" :"#c7c7c7", "NA" : "#c7c7c7", "Uncharacterized" : "#c7c7c7", "root": "#000000",
                  "TYR": "#bd065f", "CMGC": "#d5c203", "TKL": "#997e73","STE": "#80b412", # kinase groups 
                  "CK1": "#0dbae9", "AGC": "#00bba1", "CAMK":  "#1f6ed4", "NEK": "#8ce4fa", "RGC":"#f59a62"}

# Generate all parameter combinations
keys = list(grid.keys())
values = list(grid.values())
combinations = list(itertools.product(*values))

for combo in combinations:
    params = dict(zip(keys, combo))

    # Create a folder for each combination
    folder_name = "_".join([f"{k}{v}" for k, v in params.items()])
    folder_output = os.path.join(output_base, folder_name)
    os.makedirs(folder_output, exist_ok=True)
    print(f"\n--- Running projection for {folder_name} ---")

    # 1. Run Poincaré map
    cmd = build_command(params, folder_output + "/")
    print("CMD:", cmd)
    os.system(cmd)

    # 2. Load projection
    projection_file = load_projection(folder_output, params)
    if not os.path.exists(projection_file):
        print(f"Projection file not found: {projection_file}")
        continue

    df_emb = read_embeddings(projection_file, path_annotation_csv, withroot=False)

    # 3. Generate and display interactive plot for each parameter combination
    if data_type == "plm":
        title = f"Poincaré projection with embeddings: {folder_name}"
    elif data_type == "plm_aae":
        title = f"Poincaré projection with AAE embeddings: {folder_name}"
    elif data_type == "pssm":
        title = f"Poincaré projection without embeddings: {folder_name}"
    else:
        title = f"Poincaré projection with RFA matrix: {folder_name}"

    # Create and display the interactive plot
    fig = plot_embedding_interactive(
        df_emb,
        labels_name="1_Group",
        color_palette=kinase_palette,
        title=title,
        fontsize=11
    )
    fig.show()

    # Save the plot to HTML file
    output_html = os.path.join(folder_output, f"projection_{folder_name}.html")
    fig.write_html(output_html)
    print(f"✔ Saved projection to → {output_html}")

    # 4. Calculate quality metrics
    true_labels = df_emb["1_Group"].values
    coord_low = df_emb[["pm1", "pm2"]].values

    # Load original high-dimensional data
    if data_type in ["plm", "plm_aae"]:
        embedding_files = [f for f in os.listdir(embedding_paths[data_type]) if f.endswith('.pt')]
        features = []
        for file in sorted(embedding_files):
            embedding = torch.load(os.path.join(embedding_paths[data_type], file))['embedding']
            features.append(np.mean(embedding, axis=0))
        features = np.array(features)
    elif data_type == "pssm":
        pssm_files = [f for f in os.listdir("examples/kinases/") if f.endswith('.aamtx')]
        features = []
        for file in sorted(pssm_files):
            pssm = np.loadtxt(os.path.join("examples/kinases/", file)).reshape(-1)
            features.append(pssm)
        features = np.array(features)

    # Calculate all quality metrics
    try:
        model = PoincareMaps(coord_low)
        model.get_distances()
        D_proj = model.distances

        # Detect clusters
        clusters = detect_cluster(D_proj, n_clusters=len(np.unique(true_labels)))
        ARI, FMS = get_scores(true_labels, clusters)

        # Calculate Q metrics
        Qlocal, Qglobal, Kmax, df_Q = get_quality_metrics(
            coord_high=features,
            coord_low=coord_low,
            distance="poincare",
            setting="manifold",
            k_neighbours=5
        )

        # Store all results
        result = {
            "params": params,
            "ARI": ARI,
            "FMS": FMS,
            "Qlocal": Qlocal,
            "Qglobal": Qglobal,
            "Qnx": df_Q.Qnx.to_numpy() if hasattr(df_Q, 'Qnx') else np.zeros(5),
            "projection_file": projection_file,
        }
    except Exception as e:
        print(f"Error calculating metrics: {e}")
        # Store partial results if metrics calculation fails
        result = {
            "params": params,
            "ARI": ARI if 'ARI' in locals() else None,
            "FMS": FMS if 'FMS' in locals() else None,
            "Qlocal": None,
            "Qglobal": None,
            "Qnx": np.zeros(5),
            "projection_file": projection_file,
        }

    results.append(result)

print("\n=== Grid search complete ===")

=== Starting grid search ===

--- Running projection for knn5_gamma0.5_sigma1_cospca0_epochs10_seed0_distancecosine ---
CMD: PYTHONPATH=/home/hugo/Bureau/PoincareMSA:$PYTHONPATH python scripts/build_poincare_map/main.py --input_path embeddings/aae_embeddings/ankh_base_kinases/ --output_path grid_search_results/plm_aae/knn5_gamma0.5_sigma1_cospca0_epochs10_seed0_distancecosine/ --plm_embedding True --matrices_output_path grid_search_results/plm_aae/knn5_gamma0.5_sigma1_cospca0_epochs10_seed0_distancecosine//matrices/ --distlocal cosine --gamma 0.5 --pca 0 --epochs 10 --seed 0 --knn 5 
CUDA: True
Random seed set as 0
497 proteins found in folder embeddings/aae_embeddings/ankh_base_kinases/.
['416.pt', '379.pt', '458.pt', '230.pt', '53.pt', '450.pt', '367.pt', '6.pt', '150.pt', '451.pt', '259.pt', '381.pt', '236.pt', '188.pt', '235.pt', '21.pt', '117.pt', '71.pt', '193.pt', '52.pt', '399.pt', '371.pt', '95.pt', '3.pt', '377.pt', '330.pt', '443.pt', '327.pt', '444.pt', '284.pt', '391.pt', 

loss: 7.95474: 100%|████████████████████████████| 10/10 [00:01<00:00,  5.89it/s]
Ignoring fixed x limits to fulfill fixed data aspect with adjustable data limits.
Ignoring fixed x limits to fulfill fixed data aspect with adjustable data limits.


PM computed in 1.70 sec

loss = 7.955e+00
time = 0.034 min


✔ Saved projection to → grid_search_results/plm_aae/knn5_gamma0.5_sigma1_cospca0_epochs10_seed0_distancecosine/projection_knn5_gamma0.5_sigma1_cospca0_epochs10_seed0_distancecosine.html


KeyError: 'embedding'

## Results

In [ ]:
# Call comparison function
compare_projections(results)


=== Comparison of Projections ===
                                         params_base  seed       ARI  \
0  knn5_gamma0.5_sigma1_cospca0_epochs10_distance...     0  0.314860   
1  knn5_gamma0.5_sigma1_cospca0_epochs10_distance...     1  0.485848   
2  knn5_gamma0.5_sigma1_cospca0_epochs10_distance...     2  0.348309   
3  knn5_gamma1_sigma1_cospca0_epochs10_distanceco...     0  0.330104   
4  knn5_gamma1_sigma1_cospca0_epochs10_distanceco...     1  0.486620   
5  knn5_gamma1_sigma1_cospca0_epochs10_distanceco...     2  0.502331   

        FMS    Qlocal   Qglobal  
0  0.506647  0.133852  0.567047  
1  0.600891  0.156665  0.565226  
2  0.534497  0.165036  0.576612  
3  0.515184  0.140809  0.567645  
4  0.601945  0.159841  0.566397  
5  0.611205  0.166306  0.572846  
